In [1]:
"""
UNIFIED SECTOR ROUTER
Master orchestrator for multi-sector financial research
"""

import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SerpAPIWrapper
from langgraph.graph import StateGraph, END
from typing import TypedDict, List, Optional
import json
from datetime import datetime

# Load environment variables
load_dotenv()

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
search = SerpAPIWrapper()

print("✅ Unified Router Initialized!")
print("🎯 Supports: IT Sector, Pharma Sector, General Financial")


✅ Unified Router Initialized!
🎯 Supports: IT Sector, Pharma Sector, General Financial


In [2]:
# We'll import the agent functions from previous notebooks
# For now, we'll redefine them here in a unified way

# Unified Research State
class UnifiedResearchState(TypedDict):
    query: str
    detected_sector: str
    research_plan: List[str]
    current_step: int
    research_data: List[dict]
    final_report: Optional[str]

print("✅ Unified State Defined!")


✅ Unified State Defined!


In [3]:
def detect_sector(state: UnifiedResearchState) -> UnifiedResearchState:
    """Intelligently detect which sector the query belongs to"""
    
    query = state["query"]
    
    prompt = f"""You are a financial sector classification expert.

User Query: {query}

Analyze this query and determine which sector it belongs to:
- IT: Information technology, software, tech companies, AI/ML, cloud computing
- PHARMA: Pharmaceutical, drugs, biotech, healthcare, clinical trials, FDA
- GENERAL: Banking, finance, other industries, cross-sector

Respond with ONLY ONE WORD: IT, PHARMA, or GENERAL
"""
    
    response = llm.invoke(prompt)
    sector = response.content.strip().upper()
    
    # Validate sector
    if sector not in ["IT", "PHARMA", "GENERAL"]:
        sector = "GENERAL"
    
    state["detected_sector"] = sector
    
    print("\n" + "="*60)
    print("🎯 SECTOR DETECTION")
    print("="*60)
    print(f"Query: {query}")
    print(f"Detected Sector: {sector}")
    print("="*60)
    
    return state

print("✅ Sector Detection Node Ready!")


✅ Sector Detection Node Ready!


In [4]:
def generate_it_research_plan(state: UnifiedResearchState) -> UnifiedResearchState:
    """Generate IT-specific research plan"""
    
    query = state["query"]
    
    prompt = f"""You are an IT sector research expert.

User Query: {query}

Generate 7-10 research topics for IT sector analysis.
Focus on: technology trends, AI/ML, cloud, market dynamics, key players, innovations.

Return ONLY a valid JSON array: ["topic1", "topic2", ...]
"""
    
    response = llm.invoke(prompt)
    content = response.content.strip()
    
    if content.startswith("```"):
        content = content.split("```")[1]
        if content.startswith("json"):
            content = content[4:]
        content = content.strip()
    
    try:
        research_plan = json.loads(content)
    except:
        research_plan = [
            f"Overview of {query}",
            "Key IT companies and market share",
            "Technology trends and innovations",
            "Competitive landscape",
            "Financial performance metrics",
            "Future outlook and projections",
            "Regulatory and policy impacts"
        ]
    
    state["research_plan"] = research_plan
    state["current_step"] = 0
    
    print("\n📋 IT RESEARCH PLAN:")
    for i, topic in enumerate(research_plan, 1):
        print(f"{i}. {topic}")
    
    return state

print("✅ IT Research Plan Generator Ready!")


✅ IT Research Plan Generator Ready!


In [5]:
def generate_pharma_research_plan(state: UnifiedResearchState) -> UnifiedResearchState:
    """Generate Pharma-specific research plan"""
    
    query = state["query"]
    
    prompt = f"""You are a pharmaceutical industry research expert.

User Query: {query}

Generate 7-10 research topics for pharma sector analysis.
Focus on: drug pipelines, FDA approvals, clinical trials, patents, market dynamics.

Return ONLY a valid JSON array: ["topic1", "topic2", ...]
"""
    
    response = llm.invoke(prompt)
    content = response.content.strip()
    
    if content.startswith("```"):
        content = content.split("```")[1]
        if content.startswith("json"):
            content = content[4:]
        content = content.strip()
    
    try:
        research_plan = json.loads(content)
    except:
        research_plan = [
            f"Overview of {query}",
            "Drug pipeline analysis",
            "Recent FDA/regulatory approvals",
            "Clinical trial outcomes",
            "Market dynamics and competition",
            "Manufacturing capabilities",
            "Future outlook and R&D investments"
        ]
    
    state["research_plan"] = research_plan
    state["current_step"] = 0
    
    print("\n📋 PHARMA RESEARCH PLAN:")
    for i, topic in enumerate(research_plan, 1):
        print(f"{i}. {topic}")
    
    return state

print("✅ Pharma Research Plan Generator Ready!")


✅ Pharma Research Plan Generator Ready!


In [6]:
def generate_general_research_plan(state: UnifiedResearchState) -> UnifiedResearchState:
    """Generate general financial research plan"""
    
    query = state["query"]
    
    prompt = f"""You are a financial research expert.

User Query: {query}

Generate 7-10 research topics for general financial analysis.

Return ONLY a valid JSON array: ["topic1", "topic2", ...]
"""
    
    response = llm.invoke(prompt)
    content = response.content.strip()
    
    if content.startswith("```"):
        content = content.split("```")[1]
        if content.startswith("json"):
            content = content[4:]
        content = content.strip()
    
    try:
        research_plan = json.loads(content)
    except:
        research_plan = [
            f"Overview of {query}",
            "Market analysis and trends",
            "Key players and competition",
            "Financial performance",
            "Regulatory environment",
            "Future outlook",
            "Risk analysis"
        ]
    
    state["research_plan"] = research_plan
    state["current_step"] = 0
    
    print("\n📋 GENERAL RESEARCH PLAN:")
    for i, topic in enumerate(research_plan, 1):
        print(f"{i}. {topic}")
    
    return state

print("✅ General Research Plan Generator Ready!")


✅ General Research Plan Generator Ready!


In [7]:
def route_to_planner(state: UnifiedResearchState) -> str:
    """Route to appropriate research planner based on sector"""
    sector = state["detected_sector"]
    
    if sector == "IT":
        return "it_plan"
    elif sector == "PHARMA":
        return "pharma_plan"
    else:
        return "general_plan"

print("✅ Router Logic Ready!")


✅ Router Logic Ready!


In [8]:
def unified_deep_research(state: UnifiedResearchState) -> UnifiedResearchState:
    """Execute deep research regardless of sector"""
    
    current_step = state["current_step"]
    research_plan = state["research_plan"]
    
    if current_step >= len(research_plan):
        return state
    
    current_topic = research_plan[current_step]
    sector = state["detected_sector"]
    
    print(f"\n🔬 Researching [{sector}]: {current_topic}")
    
    # Initial search
    search_query = f"{current_topic} {sector.lower()} India 2025-2026"
    search_results = search.run(search_query)
    
    # Generate follow-up
    analysis_prompt = f"""Analyze this data and generate 2 follow-up search queries.

Topic: {current_topic}
Data: {search_results[:800]}

Return ONLY a JSON array: ["query1", "query2"]
"""
    
    response = llm.invoke(analysis_prompt)
    content = response.content.strip()
    
    if content.startswith("```"):
        content = content.split("```")[1]
        if content.startswith("json"):
            content = content[4:]
        content = content.strip()
    
    try:
        follow_ups = json.loads(content)
    except:
        follow_ups = [f"{current_topic} latest 2026"]
    
    all_data = [{"query": current_topic, "results": search_results}]
    
    for fq in follow_ups[:2]:
        print(f"  ↳ {fq}")
        result = search.run(fq)
        all_data.append({"query": fq, "results": result})
    
    state["research_data"].append({
        "topic": current_topic,
        "step": current_step,
        "data": all_data
    })
    
    state["current_step"] += 1
    return state

def check_research_complete(state: UnifiedResearchState) -> str:
    """Check if research is complete"""
    if state["current_step"] >= len(state["research_plan"]):
        return "generate_report"
    return "continue_research"

print("✅ Unified Deep Research Ready!")


✅ Unified Deep Research Ready!


In [9]:
def generate_unified_report(state: UnifiedResearchState) -> UnifiedResearchState:
    """Generate sector-appropriate report"""
    
    print("\n📝 Generating Comprehensive Report...")
    
    sector = state["detected_sector"]
    query = state["query"]
    
    # Compile findings
    all_findings = ""
    for item in state["research_data"]:
        all_findings += f"\n\nTopic: {item['topic']}\n"
        for data in item['data']:
            all_findings += f"{data['results'][:500]}\n"
    
    # Sector-specific report structure
    if sector == "IT":
        structure = """
1. EXECUTIVE SUMMARY
2. IT MARKET OVERVIEW
3. TECHNOLOGY TRENDS & INNOVATIONS
4. COMPETITIVE LANDSCAPE
5. AI/ML ADOPTION
6. FUTURE OUTLOOK
7. KEY RECOMMENDATIONS
"""
    elif sector == "PHARMA":
        structure = """
1. EXECUTIVE SUMMARY
2. MARKET OVERVIEW
3. R&D & DRUG PIPELINE
4. REGULATORY ENVIRONMENT
5. COMPETITIVE ANALYSIS
6. FUTURE OUTLOOK
7. KEY INSIGHTS
"""
    else:
        structure = """
1. EXECUTIVE SUMMARY
2. MARKET ANALYSIS
3. KEY PLAYERS
4. FINANCIAL PERFORMANCE
5. REGULATORY LANDSCAPE
6. FUTURE PROJECTIONS
7. RECOMMENDATIONS
"""
    
    report_prompt = f"""You are a {sector} sector analyst.

Query: {query}
Sector: {sector}

Research Findings:
{all_findings[:8000]}

Write a comprehensive report (1500-2000 words) with this structure:
{structure}

Use professional language, specific data, company names, and numbers.
"""
    
    response = llm.invoke(report_prompt)
    final_report = response.content
    
    state["final_report"] = final_report
    
    # Save report
    os.makedirs("outputs/reports", exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"outputs/reports/{timestamp}_{sector}_{query[:40].replace(' ', '_')}.md"
    
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(f"# {sector} Sector Research Report\n\n")
        f.write(f"**Query:** {query}\n\n")
        f.write(f"**Sector:** {sector}\n\n")
        f.write(f"**Date:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        f.write("---\n\n")
        f.write(final_report)
    
    print(f"\n💾 Report saved: {filename}")
    
    return state

print("✅ Unified Report Generator Ready!")


✅ Unified Report Generator Ready!


In [10]:
# Build unified workflow
workflow = StateGraph(UnifiedResearchState)

# Add all nodes
workflow.add_node("detect", detect_sector)
workflow.add_node("it_plan", generate_it_research_plan)
workflow.add_node("pharma_plan", generate_pharma_research_plan)
workflow.add_node("general_plan", generate_general_research_plan)
workflow.add_node("research", unified_deep_research)
workflow.add_node("report", generate_unified_report)

# Build the flow
workflow.set_entry_point("detect")

# Route from detection to appropriate planner
workflow.add_conditional_edges(
    "detect",
    route_to_planner,
    {
        "it_plan": "it_plan",
        "pharma_plan": "pharma_plan",
        "general_plan": "general_plan"
    }
)

# All planners go to research
workflow.add_edge("it_plan", "research")
workflow.add_edge("pharma_plan", "research")
workflow.add_edge("general_plan", "research")

# Research loops or goes to report
workflow.add_conditional_edges(
    "research",
    check_research_complete,
    {
        "continue_research": "research",
        "generate_report": "report"
    }
)

workflow.add_edge("report", END)

# Compile
unified_agent = workflow.compile()

print("✅ Unified Multi-Sector Agent Compiled!")
print("🎯 Ready to handle IT, Pharma, and General queries!")


✅ Unified Multi-Sector Agent Compiled!
🎯 Ready to handle IT, Pharma, and General queries!


In [11]:
def research_any_query(query: str):
    """
    Universal research function - handles ANY financial query!
    """
    
    print("\n" + "="*70)
    print("🚀 UNIFIED FINANCIAL RESEARCH AGENT")
    print("="*70)
    print(f"Query: {query}\n")
    
    # Initialize state
    initial_state = {
        "query": query,
        "detected_sector": "",
        "research_plan": [],
        "current_step": 0,
        "research_data": [],
        "final_report": None
    }
    
    # Execute full workflow
    final_state = unified_agent.invoke(initial_state)
    
    print("\n🎉 RESEARCH COMPLETE!")
    print(f"📊 Sector: {final_state['detected_sector']}")
    print(f"📄 Report saved to outputs/reports/")
    
    return final_state

print("✅ Universal Research Function Ready!")
print("\n🎉 UNIFIED AGENT FULLY OPERATIONAL!")


✅ Universal Research Function Ready!

🎉 UNIFIED AGENT FULLY OPERATIONAL!


In [12]:
# Test 1: IT Query
print("\n" + "="*70)
print("TEST 1: IT SECTOR")
print("="*70)
it_result = research_any_query("Analyze TCS cloud computing strategy and AI investments")

print("\n\n" + "="*70)
print("TEST 2: PHARMA SECTOR")
print("="*70)
pharma_result = research_any_query("Dr Reddy's generic drug pipeline and US market presence")

print("\n\n" + "="*70)
print("TEST 3: GENERAL FINANCIAL")
print("="*70)
general_result = research_any_query("HDFC Bank financial performance and digital banking initiatives")



TEST 1: IT SECTOR

🚀 UNIFIED FINANCIAL RESEARCH AGENT
Query: Analyze TCS cloud computing strategy and AI investments


🎯 SECTOR DETECTION
Query: Analyze TCS cloud computing strategy and AI investments
Detected Sector: IT

📋 IT RESEARCH PLAN:
1. Impact of AI/ML on Cloud Computing Strategies in IT Firms
2. Market Dynamics of Cloud Services: Key Players and Competitive Strategies
3. Innovations in Cloud Security: Trends and Best Practices
4. The Role of AI in Enhancing Cloud Infrastructure Efficiency
5. Evaluating TCS's Position in the Global Cloud Computing Market
6. Emerging Trends in Hybrid Cloud Solutions and Their Implications
7. The Future of AI-Driven Cloud Services: Opportunities and Challenges
8. Case Studies of Successful AI Implementations in Cloud Environments
9. Analyzing the Interplay Between Cloud Adoption and Digital Transformation
10. Investment Trends in AI Technologies Among Leading IT Companies

🔬 Researching [IT]: Impact of AI/ML on Cloud Computing Strategies in IT F